In [357]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [358]:
import math
import numpy as np
import json
from copy import deepcopy
import bisect

from config import filename
print(filename)

data/smallRandom.json


In [359]:
def n2letter(n):
    '''0 to 'a', 1 to 'b', ... '''
    return chr(96+n)

def string2duration(string):
    ''' "01:50:19.3177493" to duration in seconds'''
    return 3600*int(string[:2]) + 60*int(string[3:5]) + int(string[6:8])   #Duration is int

In [360]:
def read_data(path):
    global task_count
    global tasks
    file = open(path)
    data = json.load(file)
    nodes = data['nodes']
    tasks = dict()
    for task_str, info in nodes.items():
        task = int(task_str)
        tasks[task] = {'Data' : string2duration(info['Data']), 'Dependencies' : info['Dependencies']}
    task_count = len(tasks)
    print("Data loaded successfully. Number of tasks: " + str(task_count))

n_cores = 2
read_data(filename)
# tasks

Data loaded successfully. Number of tasks: 10


In [361]:
#Tasks to child tasks / Tasks to parents / Task is terminal / Task is inital
task2childs = {task : list() for task in tasks}
task2parents = {task : list() for task in tasks}
for task, info in tasks.items():
    #Add childs
    list_task_parents = info['Dependencies']
    for task_parent in list_task_parents:
        task2childs[task_parent].append(task)
    #Add parents
    task2parents[task] = tasks[task]['Dependencies']
    
def task_is_terminal(task: int):
    return len(task2childs[task]) == 0
def task_is_inital(task: int):
    return len(task2parents[task]) == 0

# print(task2childs)
# print(task2parents)

In [362]:
task2sbl = {}

def save_static_bottom_level(task : int):
    task_duration = tasks[task]["Data"]
    if task_is_terminal(task):
        sbl = task_duration
    else:
        list_sbl_child = list()
        for task_child in task2childs[task]:
            if task_child in task2sbl:
                sbl_child = task2sbl[task_child]
            else:
                sbl_child = save_static_bottom_level(task_child)
            list_sbl_child.append(sbl_child)
        sbl = max(list_sbl_child) + task_duration
                
    task2sbl[task] = sbl
    return sbl

for task in tasks:
    if task_is_inital(task):
        save_static_bottom_level(task)
        
# task2sbl



In [363]:
class Core():
    def __init__(self, core_n):
        self.core_n = core_n
        self.task = -1
        self.task_end_time = 0
    
    def __repr__(self):
        return f"({self.core_n}: task {self.task} end at {self.task_end_time})"    
    
    def __eq__(self, node):
        return self.task == node.task and self.task_end_time == node.task_end_time


In [364]:
class Node():
    def __init__(self, parent = None, task_to_add = None, core_where_to_add = None, time_task_start = None):
        '''Create a Node object ie a partial scheduling
        parent = parent Node, None if root
        task_to_add : task added to the partial schedule
        core_where_to_add : core where to do task
        time_task_start : instant where the core will start computing the task
        '''
        self.tasks_graph = tasks
        
        if parent is None:
            self.parent = None
            self.tasks_done = set()
            self.cores = {core_n : {"task" : -1, "task_end_time" : 0} for core_n in range(n_cores)}
                                  
            self.hist = ''  
            
        else:
            self.parent = parent
            self.tasks_done = deepcopy(parent.tasks_done)
            self.tasks_done.add(task_to_add)

            self.cores = deepcopy(parent.cores)
            duration_task = self.tasks_graph[task_to_add]['Data']            
            self.cores[core_where_to_add] = {"task" : task_to_add, "task_end_time" : time_task_start + duration_task}
                
            self.hist = parent.hist + f"|{core_where_to_add}{n2letter(task_to_add)}"
                 
    def __repr__(self):
        string = '[' + ','.join([n2letter(task) for task in self.tasks_done]) + ']'
        string += ''.join([f"({core['task']} end at {core['task_end_time']})" for core in self.cores.values()])
        return string
            
    def is_terminal(self):
        '''Return whether a node is a full schedule'''
        return len(self.tasks_done) == task_count
    
    def succ(self):                     
        '''Create and return list of child node of self'''
        childs = list()
        
        #On regarde toutes les tâches qu'on va tenter de rajouter
        for task, info in self.tasks_graph.items():
            
            #On passe les taches déjà ajoutées
            if task in self.tasks_done: 
                continue
            
            #On ne garde que les taches dont toutes les dépendances ont été réalisées
            if not all([task_required in self.tasks_done for task_required in info['Dependencies']]): 
                continue
            
            #On calcul le temps ou toutes les dépendances de task seront terminés par les coeurs            
            time_all_dependencies_done = 0
            for core in self.cores.values():
                #Pour chaque coeur, si la task actuellement calculée est une dépendance, on ne pourra commencer à calculer la nouvelle task qu'une fois que la task dépendante sera finie
                if core["task"] in info['Dependencies']:
                    time_all_dependencies_done = max(core["task_end_time"], time_all_dependencies_done)
                    
            for core_n, core in self.cores.items():
                #On ne commence à faire la task que lorsque toutes les dépendances sont calculées et que le core est disponible.
                time_core_available = core["task_end_time"]
                time_task_start = max(time_all_dependencies_done, time_core_available)
                child = Node(parent = self, task_to_add=task, core_where_to_add=core_n, time_task_start=time_task_start)
                childs.append(child)
                
        return childs
        
    def cost(self, child_node):
        '''Return the cost of going from self to child_node, a child node of self
        '''
        g_self = max([core["task_end_time"] for core in self.cores.values()])
        g_child = max([core["task_end_time"] for core in child_node.cores.values()])
        cost = g_child - g_self
        return cost
    
    #Node-schedule method
    def __eq__(self, node):
        '''Return whether a node is equal to another. Two nodes are considered equal if they have completed the same tasks and if all their cores stop working at same time.
        '''
        return self.tasks_done == node.tasks_done and self.set_of_core() == node.set_of_core()
    
    def set_of_core(self):
        return set([(core["task"], core["task_end_time"]) for core in self.cores.values()])
    
    def compute_g(self):
        return max([core["task_end_time"] for core in self.cores.values()])
    
root = Node()
x, y = root.succ()

# print(a, b, c, d, e, f, sep = '\n')
# print()
# print(g, h, i, j, k, l, sep = '\n')

In [365]:
def h(node):
    '''Estimated remaining time of the node-schedule for reaching a terminal node.
    Must understimate true value.'''
    successor_tasks = list()
    for task, info in tasks.items():
        if task in node.tasks_done: #On passe les taches déjà ajoutées
            continue
        if not all([task_required in node.tasks_done for task_required in info['Dependencies']]):   #On ne garde que les taches dont toutes les dépendances ont été réalisées
            continue
        successor_tasks.append(task)
    if successor_tasks == []:
        return 0
    return max([task2sbl[task] for task in successor_tasks])

h(root)

18043

In [366]:
import time

class A_star():
    def __init__(self, root):
        self.root = root
        
    def insert_in_open_list(self, node, open_list):
        idx = 0
        while idx < len(open_list):
            if node.f < open_list[idx].f:
                open_list.insert(idx, node)
                return
            idx += 1
        open_list.append(node)
        return
            
    def find_best_path(self):
        root.g = 0
        root.f = h(root)
        OPEN = [self.root]
        CLOSED = []
        
        while len(OPEN) > 0:
            current = OPEN.pop(0)
            CLOSED.append(current)
            
            if current.is_terminal():
                return current

            print(len(OPEN), end = '\r')
            for child in current.succ():
                
                if child in CLOSED:
                    continue
                
                if child in OPEN:
                    continue    #For our particular problem, the self.g will always be the same (the parent does not matter) so pass
                
                else:
                    # child.g = current.g + current.cost(child)     #General method. In our case, node.g does not depend on the path and the parent node
                    child.g = child.compute_g()
                    child.f = child.g + h(child)
                    self.insert_in_open_list(child, OPEN)
            
        raise Exception("No path from root to a terminal node")
    
root = Node()
a_star = A_star(root = root)
a_star.find_best_path()

[a,b,c,d,e,f,g,h,i,j](3 end at 20784)(6 end at 18412)